In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Hierarchical clustering

In [ ]:
customers_ml_data = pd.read_csv('data/online_retail_afterEDA.csv', index_col='CustomerID')
non_binary_cols = [
    'balance', 'max_spent', 'mean_spent', 
    'min_spent', 'n_orders','total_items', 
    'total_refunded', 'total_spent' ]

customers = customers_ml_data[non_binary_cols]
Xscores = pd.read_csv('data/pca_scores.csv', index_col='CustomerID')

In [ ]:
# This function generates a pairplot enhanced with the result of k-means
def pairplot_cluster(df, cols, cluster_assignment):
    """
    Input
        df, dataframe that contains the data to plot
        cols, columns to consider for the plot
        cluster_assignments, cluster asignment returned 
        by the clustering algorithm
    """
    # seaborn will color the samples according to the column cluster
    df['cluster'] = cluster_assignment 
    sns.pairplot(df, vars=cols, hue='cluster')
    df.drop('cluster', axis=1, inplace=True)

## Hierarchical clustering: Linking with Linkage

The main idea behind hierarchical clustering is that you start with each point in it's own cluster and then

1. compute distances between all clusters
2. merge the clusters which are closest

Do this repeatedly until you have only one cluster.

This algorithm groups the samples in a bottom-up fashion and falls under the category of agglomerative clustering algorithms.

Depending on which distance metric is used to calculate distance between clusters and samples, the clusters will have different properties. In this section we'll use a distance metric that will minimizes the variance of the clusters being merged.

This algorithm results in a hierarchy, or binary tree, of clusters branching down to the last layer which has a leaf for each point in the dataset that can be visualise with a "Dendrogram". The advantage of this approach is that clusters can grow according to the shape of the data rather than being globular.

sklearn implements hierarchical clustering in the class [`sklearn.cluster.AgglomerativeClustering`](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering), this class is mainly a wrapper to the functions in [`scipy.cluster.hierarchy`](http://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html).

### Learning Activity - Plotting dendograms
Use the function `linkage()` from `scipy.cluster.hierarchy` to cluster the retail data and pass the result to the function `dendrogram()` to visualise the result. Truncate the dendrogram if the initial result is unreadable.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

# Apply hierarchical clustering 
Z = linkage(customers, method='ward', metric='euclidean')

# Draw the dendrogram
plt.figure(figsize=(18,6))
dendrogram(Z)
plt.ylabel('Distance')
plt.xlabel('Sample index')


The coloring of the figure highlights that the data can be segmented in two big clusters that were merged only in the very last iterations of the algorithm.

We can improve the readability of the dendrogram showing only the last merged clusters and a threshold to color the clusters. For this use:

* the option `truncate_mode` in `dendrogram`.

In [ ]:
# Draw the dendrogram using a cut_off value
plt.figure(figsize=(18,6))
cut_off = 60

dendrogram(Z, color_threshold=cut_off, truncate_mode='lastp', p=20)

plt.ylabel('Distance')
plt.xlabel('Sample index or (cluster size)')
plt.hlines(cut_off, 0, len(customers), linestyle='--')


oh yeah that's a lot better!

### Learning Activity - Running Linkage with Sklearn

Use `sklearn.cluster.AgglomerativeClustering` to cluster the retail data according to the 3 clusters highlighted by the dendrogram above and visualise the result in the subspace give by the features `mean_spent` and `max_spent`.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Perform clustering with AgglomerativeClustering
# Ward merges two clusters if the resulting has small variance

n_cluster = 3
agglomerative = AgglomerativeClustering(
                    n_clusters=n_cluster, linkage='ward', 
                    affinity='euclidean')

cluster_assignment = agglomerative.fit_predict(customers)


Let's visualise this with the `pairplot_cluster()`

In [ ]:
# Visualise the clusters using pairplot_cluster()
pairplot_cluster(customers, ['mean_spent', 'max_spent'], cluster_assignment)
